In [1]:
import weaviate
import weaviate.classes as wvc
import os
import requests
import json
from dotenv import load_dotenv
from weaviate.classes.config import Configure, Property, DataType, Tokenization
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import Tensor
import torch.nn.functional as F

load_dotenv()
openai_api_key = os.getenv("OPENAI_KEY")

client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=weaviate.config.AdditionalConfig(timeout=(60, 180)),
    headers={
        "X-OpenAI-Api-Key": openai_api_key  # Replace with your inference API key
    }
)

/root/miniconda3/envs/m_enmb/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

In [3]:
path_to_pdf = 'pdf_docs'

documents_text = []

for doc in os.listdir(path_to_pdf):

    doc_path = f'{path_to_pdf}/{doc}'
    loader = PyPDFLoader(doc_path)
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
    docs = text_splitter.split_documents(pages)
    documents_text.append(docs)

documents_text = [item for sublist in documents_text for item in sublist]

In [4]:
import torch
from transformers import BitsAndBytesConfig, AutoModel

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

encoder = AutoModel.from_pretrained(
    'Salesforce/SFR-Embedding-Mistral',
    trust_remote_code=True,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.87s/it]


In [5]:
for d in documents_text:
    d = d
    break

source = d.metadata['source']
page = str(d.metadata['page'])
content = d.page_content

In [6]:
tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral')
max_length = 4096
body_vectors = []
title_vectors = []

for d in documents_text:

    body = d.page_content
    title = d.metadata['source']
    body_batch_dict = tokenizer(body, max_length=max_length, padding=True, truncation=True, return_tensors="pt").to('cuda')
    title_batch_dict = tokenizer(title, max_length=max_length, padding=True, truncation=True, return_tensors="pt").to('cuda')
    body_outputs = encoder(**body_batch_dict)
    title_outputs = encoder(**title_batch_dict)
    body_embs = last_token_pool(body_outputs.last_hidden_state, body_batch_dict['attention_mask']).float().cpu().detach().numpy(force=True)
    title_embs = last_token_pool(title_outputs.last_hidden_state, title_batch_dict['attention_mask']).float().cpu().detach().numpy(force=True)

    body_vectors.append(body_embs)
    title_vectors.append(title_embs)
    torch.cuda.empty_cache()